In [9]:
import streamlit as st
import requests
import pymongo
import pandas as pd
import base64
import os
from langchain_groq import ChatGroq
import re
from dotenv import load_dotenv
from neo4j import GraphDatabase
from PIL import Image
from io import BytesIO
from langchain_ollama import OllamaLLM
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_community import GoogleSearchAPIWrapper

load_dotenv()
# Styling

True

In [3]:
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASSWORD"))
)


In [5]:
client = pymongo.MongoClient(os.getenv("MONGO_URI"))
db = client["fracsnet"]
users_col = db["users"]
products_col = db["products"]
interactions_col = db["interactions"]
chat_history_col = db["chat_history"]

In [4]:
df = pd.read_csv("./knowledge/Cleaned_Dataset.csv").fillna("")
df["combined_text"] = df[["ProductName", "Nutrient_category","Description","Formulated_For","HealthConcern", "Benefits"]].astype(str).agg(" ".join, axis=1)

In [ ]:
# Models
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# llm = OllamaLLM(model="qwen:4b", base_url="http://localhost:11434")
llm = ChatGroq(model_name="llama-3.3-70b-versatile")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


C:\Users\mayur\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


C:\Users\mayur\AppData\Local\Temp\ipykernel_5748\1155927723.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [10]:
search = GoogleSearchAPIWrapper(
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    google_cse_id=os.getenv("GOOGLE_CSE_ID")
)

In [12]:
# Recommendation logic
def get_neo4j_recommendations(query_text, top_k=5):
    query_embedding = embedding_model.embed_query(query_text)
    keywords = set(query_text.lower().split())

    with driver.session() as session:
        results = session.run("""
            CALL db.index.vector.queryNodes('product_embeddings', $top_k, $embedding)
            YIELD node, score
            RETURN node.ProductName AS name,
                   node.Description AS description,
                   node.ProductImage AS image,
                   node.PdfLink AS pdf_link,
                   node.Benefits AS benefits,
                   node.HealthConcern AS concern,
                   score
        """, top_k=top_k * 3, embedding=query_embedding)

        filtered = []
        for record in results:
            item = dict(record)
            text_blob = " ".join([str(item.get("description", "")), str(item.get("concern", "")), str(item.get("benefits", ""))]).lower()
            overlap = keywords.intersection(set(text_blob.split()))
            adjusted_score = item["score"] + 0.5 * len(overlap)
            if overlap or item["score"] > 0.75:
                item["adjusted_score"] = adjusted_score
                filtered.append(item)

        return sorted(filtered, key=lambda x: x["adjusted_score"], reverse=True)[:top_k]

def get_user_based_recommendations(username, top_k=3):
    user_interactions = interactions_col.find({"username": username})
    product_names = [ui["product_name"] for ui in user_interactions]
    if not product_names:
        return None
    df_user = df[df["ProductName"].isin(product_names)]
    combined_keywords = " ".join(df_user["combined_text"].tolist())
    return get_neo4j_recommendations(combined_keywords, top_k=top_k)

In [11]:
def hybrid_recommendation_tool(username, user_query):
    user = users_col.find_one({"username": username})
    query_text = f"Health condition: {user_query}. Age: {user['age']}, Gender: {user['gender']}. Looking for effective supplements or natural remedies."
    hybrid = get_neo4j_recommendations(query_text)
    user_cf = get_user_based_recommendations(username)
    if user_cf:
        hybrid.extend([item for item in user_cf if item["name"] not in {c["name"] for c in hybrid}])
    seen = set()
    final = []
    for item in hybrid:
        if item["name"] not in seen:
            seen.add(item["name"])
            final.append(item)
        if len(final) >= 3:
            break
    return final or [search.run(user_query)]

In [ ]:
hybrid_recommendation_tool("saloni", "recommend me a supplement for diabetes")

TypeError: 'NoneType' object is not subscriptable

: 

In [6]:
# Core health agent functions (no Streamlit UI)
import pymongo
import pandas as pd
import base64
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from neo4j import GraphDatabase
from PIL import Image
from io import BytesIO
from langchain_ollama import OllamaLLM
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_community import GoogleSearchAPIWrapper

# Load environment
load_dotenv()

# Neo4j setup
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASSWORD"))
)

# MongoDB setup
client = pymongo.MongoClient(os.getenv("MONGO_URI"))
db = client["fracsnet"]
users_col = db["users"]
products_col = db["products"]
interactions_col = db["interactions"]
chat_history_col = db["chat_history"]

# Dataset
df = pd.read_csv("./knowledge/Cleaned_Dataset.csv").fillna("")
df["combined_text"] = df[["ProductName", "Nutrient_category", "Description", "Formulated_For", "HealthConcern", "Benefits"]].astype(str).agg(" ".join, axis=1)

# Models
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm = ChatGroq(model_name="llama-3.3-70b-versatile")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
search = GoogleSearchAPIWrapper(
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    google_cse_id=os.getenv("GOOGLE_CSE_ID")
)

# Recommendation logic
def get_neo4j_recommendations(query_text, top_k=5):
    query_embedding = embedding_model.embed_query(query_text)
    keywords = set(query_text.lower().split())

    with driver.session() as session:
        results = session.run("""
            CALL db.index.vector.queryNodes('product_embeddings', $top_k, $embedding)
            YIELD node, score
            RETURN node.ProductName AS name,
                   node.Description AS description,
                   node.ProductImage AS image,
                   node.PdfLink AS pdf_link,
                   node.Benefits AS benefits,
                   node.HealthConcern AS concern,
                   score
        """, top_k=top_k * 3, embedding=query_embedding)

        filtered = []
        for record in results:
            item = dict(record)
            text_blob = " ".join([str(item.get("description", "")), str(item.get("concern", "")), str(item.get("benefits", ""))]).lower()
            overlap = keywords.intersection(set(text_blob.split()))
            adjusted_score = item["score"] + 0.5 * len(overlap)
            if overlap or item["score"] > 0.75:
                item["adjusted_score"] = adjusted_score
                filtered.append(item)

        return sorted(filtered, key=lambda x: x["adjusted_score"], reverse=True)[:top_k]

def get_user_based_recommendations(username, top_k=3):
    user_interactions = interactions_col.find({"username": username})
    product_names = [ui["product_name"] for ui in user_interactions]
    if not product_names:
        return None
    df_user = df[df["ProductName"].isin(product_names)]
    combined_keywords = " ".join(df_user["combined_text"].tolist())
    return get_neo4j_recommendations(combined_keywords, top_k=top_k)

def hybrid_recommendation_tool(username, user_query):
    user = users_col.find_one({"username": username})
    query_text = f"Health condition: {user_query}. Age: 19, Gender: female. Looking for effective supplements or natural remedies."
    hybrid = get_neo4j_recommendations(query_text)
    user_cf = get_user_based_recommendations(username)
    if user_cf:
        hybrid.extend([item for item in user_cf if item["name"] not in {c["name"] for c in hybrid}])
    seen = set()
    final = []
    for item in hybrid:
        if item["name"] not in seen:
            seen.add(item["name"])
            final.append(item)
        if len(final) >= 3:
            break
    return final or [search.run(user_query)]

def get_product_image(product_name):
    product = products_col.find_one({"product_name": product_name}, {"image_base64": 1})
    if product and "image_base64" in product:
        try:
            decoded = base64.b64decode(product["image_base64"])
            return Image.open(BytesIO(decoded))
        except:
            return None
    return None

def record_interaction(username, product_name):
    interactions_col.insert_one({"username": username, "product_name": product_name})

def record_chat(username, query, response):
    chat_history_col.insert_one({"username": username, "query": query, "response": response})

def get_user_chat_history(username):
    return list(chat_history_col.find({"username": username}, {"_id": 0}))

def match_product_by_name(user_query):
    query_lower = user_query.lower()
    for product in df["ProductName"]:
        if product.lower() in query_lower:
            return product
    return None

# LangChain Agent Tool
def recommendation_agent_tool(input):
    username, query = input.split("|")
    recs = hybrid_recommendation_tool(username.strip(), query.strip())
    return "\n".join([r["name"] if isinstance(r, dict) else r for r in recs])

tools = [Tool(name="Recommender", func=recommendation_agent_tool, description="Use to get supplement recommendations by passing 'username|query' format")]
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=False,
    handle_parsing_errors=True
)

# Direct usage function
def ask_health_bot(username, user_query):
    product = match_product_by_name(user_query)
    if product:
        product_info = df[df["ProductName"] == product].iloc[0].to_dict()
        record_interaction(username, product)
        record_chat(username, user_query, f"Matched product: {product}")
        return {"type": "matched_product", "product": product_info}
    else:
        response = agent.run(f"{username}|{user_query}")
        results = hybrid_recommendation_tool(username, user_query)
        record_chat(username, user_query, response)
        return {"type": "recommendations", "results": results}


In [9]:
a = ask_health_bot("saloni", "recommend me a supplement for diabetes")

for product in a['results']:
    print(f"Product Name: {product['name']}")
    print(f"Description: {product['description']}\n")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: HealthConcern)} {position: line: 9, column: 25, offset: 376} for query: "\n            CALL db.index.vector.queryNodes('product_embeddings', $top_k, $embedding)\n            YIELD node, score\n            RETURN node.ProductName AS name,\n                   node.Description AS description,\n                   node.ProductImage AS image,\n                   node.PdfLink AS pdf_link,\n                   node.Benefits AS benefits,\n                   node.HealthConcern AS concern,\n                   score\n        "
Received notification from DBMS server:

Product Name: Diaxinol®
Description: Diaxinol® is a natural and synergistic formula for 
promoting balanced blood sugar levels.

Product Name: InosiCare
Description: InosiCare is a cutting-edge inositol formula that includes clinically researched 
ingredients that harmonize metabolic function, optimize hormonal balance, 
and enhance microbiome health

Product Name: Soft Tissue Support Pack
Description: A combination of four top products for optimal tissue and muscle support.

